<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/colab_cucat_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cu_cat demo

(not so risky)

In [ ]:
# !pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11==22.10.0 cudf-cu11==22.10.0 cugraph-cu11==22.10.0 pylibraft_cu11==22.10.0 raft_dask_cu11==22.10.0 dask_cudf_cu11==22.10.0 pylibcugraph_cu11==22.10.0 pylibraft_cu11==22.10.0
# !pip install git+https://github.com/graphistry/pygraphistry.git@cuCat
# !pip install -U git+https://github.com/graphistry/cu-cat.git@cu-cat #@0.01.0

In [1]:
import cuml
cuml.__version__

'22.10.00a+119.gb30ef85c1'

In [2]:
!echo "0.01.0" > VERSION.txt
!cp VERSION.txt /usr/local/lib/python3.8/dist-packages/cu_cat/

In [3]:
from importlib import reload
# reload(cu_cat)
import cu_cat
print(cu_cat.__file__)

/usr/local/lib/python3.8/dist-packages/cu_cat/__init__.py


In [4]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)


In [5]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key
graphistry.__version__


'0.28.6+17.g21dac04'

In [6]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding
# !pip install dirty_cat
import dirty_cat

In [ ]:
import logging
logger = logging.getLogger()


## ask HN

In [ ]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

# df = askHN.sample(1000,replace=True) # set smaller if you want to test a minibatch 
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()

# g1a = g.featurize(feature_engine='cuCat',memoize=False)
g1a = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat')

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cucat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         24512419 function calls (24455828 primitive calls) in 113.449 seconds

   Ordered by: internal time
   List reduced from 2487 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       13   35.883    2.760   51.176    3.937 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:984(_multiplicative_update_h_smallfast)
      242   10.592    0.044   19.350    0.080 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:932(_multiplicative_update_h)
        9    9.809    1.090    9.810    1.090 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       36    7.886    0.219    7.886    0.219 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
        1    7.176    7.176    7.404    7.404 /usr/local/lib/python3.8/dist-packages/cuml/internals/api_decorators.py:397(inner_with_setters)
       21    4.920    0.234   12.281    0.585 /usr/local/lib/python3.8/dist-packages/sklearn/feature_extr

In [ ]:
g = graphistry.nodes(askHN)
pr = cProfile.Profile()
pr.enable()


# g1b = g.featurize(feature_engine='dirty_cat',memoize=False)
g1b = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat')

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

# with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

         35746123 function calls (35683329 primitive calls) in 161.237 seconds

   Ordered by: internal time
   List reduced from 2498 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      764   44.261    0.058   93.230    0.122 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
4183584/4182799   41.336    0.000   51.035    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      360   15.234    0.042   37.327    0.104 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
        9    8.568    0.952    8.568    0.952 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
  3786716    5.616    0.000   46.662    0.000 <__array_function__ internals>:177(dot)
       21    4.458    0.212   11.217    0.534 /usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:1184(_count_vocab)
    15120 

In [ ]:
g1a.plot()

In [ ]:
g1b.plot()

## redteam

In [9]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))
tdf.RED.sum()

749.0

In [10]:
g = graphistry.nodes(tdf)
pr = cProfile.Profile()
pr.enable()

g4a = g.featurize(feature_engine='cu_cat',memoize=False)
# g3 = g.umap(engine='cuml',feature_engine='cuCat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)


         17058802 function calls (16661063 primitive calls) in 174.673 seconds

   Ordered by: internal time
   List reduced from 1760 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       30   70.795    2.360   90.281    3.009 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:984(_multiplicative_update_h_smallfast)
       18   58.701    3.261   58.702    3.261 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
       35    9.636    0.275    9.636    0.275 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
    17330    6.605    0.000    6.814    0.000 /usr/local/lib/python3.8/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
      836    3.347    0.004    3.355    0.004 {built-in method cupy._core._routines_linalg.matmul}
       42    3.260    0.078   10.184    0.242 /usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:1184(_count_vocab)
   282233    2.415    0.000

In [11]:
g = graphistry.nodes(tdf)
pr = cProfile.Profile()
pr.enable()

g4b = g.featurize(feature_engine='dirty_cat',memoize=False)
# g3 = g.umap(engine='cuml',feature_engine='cuCat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)


         103571112 function calls (103173563 primitive calls) in 369.077 seconds

   Ordered by: internal time
   List reduced from 1397 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     5948   73.869    0.012  225.811    0.038 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
12355062/12353659   58.847    0.000  115.655    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       18   55.493    3.083   55.494    3.083 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
     4650   22.036    0.005   64.763    0.014 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:971(_multiplicative_update_w)
   682577   19.362    0.000   57.572    0.000 /usr/local/lib/python3.8/dist-packages/scipy/sparse/compressed.py:136(check_format)
  1384246   16.671    0.000   37.872    0.000 /usr/local/lib/python3.8/dist-packages/scipy/sparse/spu

In [12]:
%%time
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g4a = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         30492546 function calls (30452129 primitive calls) in 113.779 seconds

   Ordered by: internal time
   List reduced from 1898 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
2944219/2943978   18.326    0.000   34.630    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     1054   17.482    0.017   50.493    0.048 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
        3   16.075    5.358   16.076    5.359 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    39524    8.782    0.000   11.134    0.000 /usr/local/lib/python3.8/dist-packages/pandas/core/missing.py:60(mask_missing)
        1    7.638    7.638    7.860    7.860 /usr/local/lib/python3.8/dist-packages/cuml/internals/api_decorators.py:397(inner_with_setters)
   134889    3.931    0.000   11.697    0.000 /usr/local/lib/python3.8/dist-packages/scipy/sparse/compressed.

In [13]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g4b = g.umap(X=['feats'], 
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            # engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         12140156 function calls (12099753 primitive calls) in 66.915 seconds

   Ordered by: internal time
   List reduced from 1890 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5   18.666    3.733   22.082    4.416 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:984(_multiplicative_update_h_smallfast)
        3   16.182    5.394   16.182    5.394 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
    39524    8.076    0.000   10.303    0.000 /usr/local/lib/python3.8/dist-packages/pandas/core/missing.py:60(mask_missing)
42595/42364    6.927    0.000   23.231    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
     4282    2.631    0.001    2.684    0.001 /usr/local/lib/python3.8/dist-packages/rmm/rmm.py:216(rmm_cupy_allocator)
    79467    1.751    0.000    1.751    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        7    1.125    0.161    3.344    0

In [14]:
g4a.plot()

In [15]:
g4b.plot()

## CTU-13 malware dataset

In [16]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



Botnet abundance: 0.69%


In [17]:
from inspect import getmodule

In [18]:
# lets umap the data
g = graphistry.edges(edf, src, dst)

pr = cProfile.Profile()
pr.enable()
g5a = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='cu_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         4576150 function calls (4435317 primitive calls) in 27.341 seconds

   Ordered by: internal time
   List reduced from 2161 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    6.021    0.669    6.021    0.669 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
        3    1.881    0.627    1.881    0.627 /usr/local/lib/python3.8/dist-packages/numpy/ma/core.py:3211(__getitem__)
       15    1.578    0.105    2.188    0.146 /usr/local/lib/python3.8/dist-packages/cu_cat/_gap_encoder.py:984(_multiplicative_update_h_smallfast)
      138    1.433    0.010    1.433    0.010 {method 'argsort' of 'numpy.ndarray' objects}
244593/188439    1.008    0.000   12.681    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      320    0.985    0.003    0.998    0.003 {built-in method cupy._core.core.array}
       46    0.889    0.019    0.889    0.019 {method 'copy' of 'numpy.ndarra

In [19]:
### lets umap the data
g = graphistry.edges(edf, src, dst)
pr = cProfile.Profile()
pr.enable()
g5b = g.umap(kind='edges', 
            X=good_cols_with_edges, 
            y = ['bot'], 
            use_scaler='quantile',
            use_scaler_target=None,
            cardinality_threshold=20,
            cardinality_threshold_target=2, 
            n_topics=n_topics,
            feature_engine='dirty_cat',
            engine='cuml',
            memoize=False,
            n_topics_target=n_topics_target,
            n_bins=n_topics_target,
            metric='euclidean', 
            n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         15585643 function calls (15444849 primitive calls) in 65.775 seconds

   Ordered by: internal time
   List reduced from 1950 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1389    7.522    0.005   25.169    0.018 /usr/local/lib/python3.8/dist-packages/dirty_cat/_gap_encoder.py:1003(_multiplicative_update_h)
      137    7.126    0.052    7.126    0.052 {method 'argsort' of 'numpy.ndarray' objects}
1673174/1616992    5.890    0.000   20.677    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        9    5.139    0.571    5.139    0.571 /usr/local/lib/python3.8/dist-packages/numpy/lib/arraysetops.py:519(in1d)
        3    5.033    1.678    5.033    1.678 /usr/local/lib/python3.8/dist-packages/numpy/ma/core.py:3211(__getitem__)
    90864    2.351    0.000    6.961    0.000 /usr/local/lib/python3.8/dist-packages/scipy/sparse/compressed.py:136(check_format)
   186528    2.035    0.000    4.66

In [20]:
g5a.plot()

In [21]:
g5b.plot()

## winlogs

### this example finds the middle ground (1e9 > samples * features > 2e9) where cpu is faster

In [ ]:
winlogs=winlogs.sample(50000,replace=False).iloc[:,:10]

In [ ]:
g3a.plot()

In [ ]:
g3b.plot()

### this large dataset uses same dirty_cat loop just passing to gpu

*   smaller examples process everything at once on gpu for speedup



In [11]:
# !wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')

winlogs=winlogs.sample(100000,replace=False).iloc[:,:10]

In [ ]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()

g3a = g.featurize(feature_engine='cu_cat')#,memoize=False)
# g3a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

# with open('winlogs_featurize_cucat_100k.txt', 'w') as stream:
#     stats = Stats(pr,stream=stream)
#     stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(winlogs)
pr = cProfile.Profile()
pr.enable()


g3b = g.featurize(feature_engine='dirty_cat',memoize=False)
# g3b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_dirtycat_50k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

## 20newsgroups

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 1000
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
g2a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
g2b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

In [ ]:
g2a.plot()

In [ ]:
g2b.plot()